<a href="https://colab.research.google.com/github/yms0606/SecuLayer/blob/main/%EC%9B%94%EA%B0%84%20%EB%8D%B0%EC%9D%B4%EC%BD%98%20%EB%B2%95%EC%9B%90%20%ED%8C%90%EA%B2%B0%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EC%9B%94%EA%B0%84_%EB%8D%B0%EC%9D%B4%EC%BD%98_%EB%B2%95%EC%9B%90_%ED%8C%90%EA%B2%B0_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
!pip install text_hammer

In [ ]:
!pip install nltk

In [7]:
import pandas as pd
import numpy as np
import random
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, PassiveAggressiveClassifier, Perceptron, LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.utils import check_random_state
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from nltk.tokenize import word_tokenize, sent_tokenize

In [8]:
import text_hammer as th
from scipy import sparse
from  tqdm import tqdm

In [24]:
train = pd.read_csv('/content/drive/MyDrive/ML study/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML study/dataset/test.csv')
train.shape

(2478, 5)

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
def count_sentences(row):
  sentences = sent_tokenize(row)
  return len(sentences)

train['num_sentences'] = train['facts'].apply(lambda x: count_sentences(x))

rows_with_single_sentence = train[train['num_sentences']== 1]

rows_with_single_sentence

,ID,first_party,second_party,facts,first_party_winner,num_sentences
248,TRAIN_0248,New York,Ferber,A New York child pornography law prohibited pe...,1,1
554,TRAIN_0554,Riverside County,McLaughlin,McLaughlin was arrested without a warrant and ...,1,1
630,TRAIN_0630,Earl R. Deen,"Gulf, Colorado & Santa Fe Railway Company",Not Available.\n,1,1
764,TRAIN_0764,Nollan,California Coastal Commission,The California Coastal Commission required own...,1,1
858,TRAIN_0858,Morrison,Olson,The Ethics in Government Act of 1978 created a...,1,1
1104,TRAIN_1104,Eastern Enterprises,Apfel,Currently unknown.\n,1,1
1285,TRAIN_1285,Smith,Daily Mail Publishing Company,A West Virginia statute made it a crime for a ...,0,1
1319,TRAIN_1319,National Association for the Advancement of Co...,Button,The NAACP was prosecuted for violating a Virgi...,1,1
1329,TRAIN_1329,United States,Paradise,In response to a series of NAACP-initiated law...,0,1
1461,TRAIN_1461,Jones,Alfred H. Mayer Company,"Jones, a black man, charged that a real estate...",1,1


In [12]:
train['facts'][86]

'The Audubon Regional library operated three branches and two bookmobiles. Blacks were not allowed to enter any of the branch libraries. The bookmobiles were segregated: a red one served only whites and a blue one served blacks. Brown was a black man who entered a library branch with four other blacks and requested a book, The Story of the Negro. The librarian informed Brown that the book was not available, but that she would request it through the state library, and he could pick it up or have it mailed to him. After the conversation, the men sat down (making no noise or disturbance) and refused to leave. They were arrested "for not leaving a public building when asked to do so by an officer."\n'

In [26]:
indexes_to_remove = [630, 1104, 1595]
train = train.drop(indexes_to_remove)
train = train.reset_index(drop=True)

In [27]:
is_not_available = train['facts'].str.contains('Not available', case = False)
is_currently_available = train['facts'].str.contains('Currently available', case = False)
is_currently_unknown = train['facts'].str.contains('Currently unknown', case = False)

train[is_not_available | is_currently_available | is_currently_unknown]

,ID,first_party,second_party,facts,first_party_winner,num_sentences
86,TRAIN_0086,Brown,Louisiana,The Audubon Regional library operated three br...,1,7
574,TRAIN_0574,Gil Garcetti et al.,Richard Ceballos,"Richard Ceballos, an employee of the Los Angel...",1,6
648,TRAIN_0649,"Friends of the Earth, Inc.","Laidlaw Environmental Services (TOC), Inc.","After Laidlaw Environmental Services, Inc. bou...",1,13
1061,TRAIN_1062,"Kenneth F. Fare, Acting Chief Probation Officer",Michael C.,"Police arrested Michael C., a 16 year old, on ...",1,9
1610,TRAIN_1613,"Atlantic Sounding Co., Inc., et al.",Edgar L. Townsend,"In July 2005, Edgar Townsend was allegedly inj...",0,13
2230,TRAIN_2233,"Verizon Communications, Inc.",Federal Communications Commission,The Telecommunications Act of 1996 entitles ne...,0,5


In [28]:
def get_clean(x):
  x = str(x).lower().replace('\\','').replace('_','')
  x = th.cont_exp(x)
  x = th.remove_emails(x)
  x = th.remove_urls(x)
  x = th.remove_html_tags(x)
  x = th.remove_rt(x)
  x = th.remove_accented_chars(x)
  x = th.remove_special_chars(x)
  x = re.sub("(.)\\1{2.}",'\\1',x)
  return x

In [29]:
def swap_labels(row):
  if row['first_party_winner'] == 1:
    row['first_party_winner'] = 0
  else:
    row['first_party_winner'] = 1
  return row

In [30]:
train = train[train['num_sentences']>=3]

In [31]:
train_swapped = train.copy()
train_swapped[['first_party','second_party']] = train_swapped[['second_party','first_party']]
train_swapped = train_swapped.apply(swap_labels, axis=1)
train_extended = pd.concat([train, train_swapped])
train_extended['facts'] = train_extended['facts'].apply(lambda x: get_clean(x))
train_extended['first_party'] = train_extended['first_party'].apply(lambda x: get_clean(x))
train_extended['second_party'] =  train_extended['second_party'].apply(lambda x: get_clean(x))
train_extended.reset_index(drop=True, inplace = True)

test['facts'] = test['facts'].apply(lambda x: get_clean(x))
test['first_party'] = test['first_party'].apply(lambda x : get_clean(x))
test['second_party'] = test['second_party'].apply(lambda x : get_clean(x))

In [27]:
train_extended

,ID,first_party,second_party,facts,first_party_winner,num_sentences
0,TRAIN_0000,phil a st amant,herman a thompson,on june 27 1962 phil st amant a candidate for ...,1,7
1,TRAIN_0001,stephen duncan,lawrence owens,ramon nelson was riding his bike when he suffe...,0,7
2,TRAIN_0002,billy joe magwood,tony patterson warden et al,an alabama state court convicted billy joe mag...,1,8
3,TRAIN_0003,linkletter,walker,victor linkletter was convicted in state court...,0,3
4,TRAIN_0004,william earl fikes,alabama,on april 24 1953 in selma alabama an intruder ...,1,9
...,...,...,...,...,...,...
4777,TRAIN_2473,renewable fuels association et al,hollyfrontier cheyenne refining llc et al,congress amended the clean air act through the...,0,5
4778,TRAIN_2474,alliance bond fund inc,grupo mexicano de desarrollo s a,alliance bond fund inc an investment fund purc...,0,7
4779,TRAIN_2475,united states,peguero,in 1992 the district court sentenced manuel d ...,1,6
4780,TRAIN_2476,st cyr,immigration and naturalization service,on march 8 1996 enrico st cyr a lawful permane...,1,8


In [32]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,3), min_df=3)

def get_vector(vectorizer, df, train_mode):
  if train_mode:
    X_facts = vectorizer.fit_transform(df['facts'])
  else:
    X_facts = vectorizer.transform(df['facts'])
  X_party1 = vectorizer.transform(df['first_party'])
  X_party2 = vectorizer.transform(df['second_party'])

  X = sparse.hstack([X_party1, X_party2, X_facts])
  return X

In [33]:
X_train = get_vector(vectorizer, train_extended, True)
Y_train = train_extended['first_party_winner']
X_test = get_vector(vectorizer, test, False)

In [36]:
train_extended.to_csv('train.csv')
test.to_csv('test.csv')

In [35]:
X_train

<4782x99897 sparse matrix of type '<class 'numpy.float64'>'
	with 572088 stored elements in Compressed Sparse Row format>

In [34]:
X_train[0][0]

<1x99897 sparse matrix of type '<class 'numpy.float64'>'
	with 94 stored elements in Compressed Sparse Row format>

In [21]:
X_train = X_train.toarray()
X_test = X_test.toarray()

In [37]:
ridge_model = RidgeClassifier()
ETC_model = ExtraTreesClassifier()
logistic_model = LogisticRegression()
RFC_model = RandomForestClassifier()
ABS_model = AdaBoostClassifier()
SGD_model = SGDClassifier()
xgb_model = XGBClassifier()
lgbm_model = LGBMClassifier()
cat_model = CatBoostClassifier()

In [38]:
ridge_model.fit(X_train, Y_train)
RFC_model.fit(X_train, Y_train)
logistic_model.fit(X_train, Y_train)

LogisticRegression()

In [39]:
ABS_model.fit(X_train, Y_train)
SGD_model.fit(X_train, Y_train)

SGDClassifier()

In [40]:
lgbm_model.fit(X_train, Y_train)
cat_model.fit(X_train, Y_train)

Learning rate set to 0.020097
0:	learn: 0.6930050	total: 294ms	remaining: 4m 53s
1:	learn: 0.6928566	total: 496ms	remaining: 4m 7s
2:	learn: 0.6926203	total: 691ms	remaining: 3m 49s
3:	learn: 0.6924690	total: 889ms	remaining: 3m 41s
4:	learn: 0.6923784	total: 1.09s	remaining: 3m 36s
5:	learn: 0.6922479	total: 1.28s	remaining: 3m 32s
6:	learn: 0.6921384	total: 1.49s	remaining: 3m 30s
7:	learn: 0.6920466	total: 1.68s	remaining: 3m 28s
8:	learn: 0.6918906	total: 1.88s	remaining: 3m 27s
9:	learn: 0.6918278	total: 2.08s	remaining: 3m 25s
10:	learn: 0.6917033	total: 2.28s	remaining: 3m 24s
11:	learn: 0.6915726	total: 2.48s	remaining: 3m 24s
12:	learn: 0.6915235	total: 2.68s	remaining: 3m 23s
13:	learn: 0.6913866	total: 2.88s	remaining: 3m 22s
14:	learn: 0.6912215	total: 3.07s	remaining: 3m 21s
15:	learn: 0.6910992	total: 3.27s	remaining: 3m 21s
16:	learn: 0.6910004	total: 3.46s	remaining: 3m 20s
17:	learn: 0.6909445	total: 3.66s	remaining: 3m 19s
18:	learn: 0.6907929	total: 3.86s	remaining: 

In [ ]:
#xgb_model.fit(X_train, Y_train)

In [41]:
voting_model = VotingClassifier(
    estimators=[('ridge',ridge_model),('ETC',ETC_model),('logistic',logistic_model),('RFC',RFC_model),
     ('ABS',ABS_model), ('SGD',SGD_model),('lgbm',lgbm_model),('cat',cat_model)],
    voting='hard'
)

voting_model.fit(X_train, Y_train)

Learning rate set to 0.020097
0:	learn: 0.6930050	total: 234ms	remaining: 3m 53s
1:	learn: 0.6928566	total: 429ms	remaining: 3m 34s
2:	learn: 0.6926203	total: 618ms	remaining: 3m 25s
3:	learn: 0.6924690	total: 809ms	remaining: 3m 21s
4:	learn: 0.6923784	total: 1s	remaining: 3m 19s
5:	learn: 0.6922479	total: 1.2s	remaining: 3m 18s
6:	learn: 0.6921384	total: 1.4s	remaining: 3m 18s
7:	learn: 0.6920466	total: 1.59s	remaining: 3m 17s
8:	learn: 0.6918906	total: 1.8s	remaining: 3m 17s
9:	learn: 0.6918278	total: 1.99s	remaining: 3m 16s
10:	learn: 0.6917033	total: 2.18s	remaining: 3m 16s
11:	learn: 0.6915726	total: 2.38s	remaining: 3m 15s
12:	learn: 0.6915235	total: 2.57s	remaining: 3m 15s
13:	learn: 0.6913866	total: 2.77s	remaining: 3m 14s
14:	learn: 0.6912215	total: 2.96s	remaining: 3m 14s
15:	learn: 0.6910992	total: 3.16s	remaining: 3m 14s
16:	learn: 0.6910004	total: 3.35s	remaining: 3m 14s
17:	learn: 0.6909445	total: 3.55s	remaining: 3m 13s
18:	learn: 0.6907929	total: 3.75s	remaining: 3m 13

VotingClassifier(estimators=[('ridge', RidgeClassifier()),
                             ('ETC', ExtraTreesClassifier()),
                             ('logistic', LogisticRegression()),
                             ('RFC', RandomForestClassifier()),
                             ('ABS', AdaBoostClassifier()),
                             ('SGD', SGDClassifier()),
                             ('lgbm', LGBMClassifier()),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7f67bbfe0d90>)])

In [42]:
submit = pd.read_csv('/content/drive/MyDrive/ML study/dataset/sample_submission.csv')

pred = voting_model.predict(X_test)

submit['first_party_winner'] = pred
submit.to_csv('submit_voting.csv', index=False)